# rubin_nights demo

The rubin_nights package provides simple utilties to accessing the EFD, the ConsDB and various logging reports. 


In [ ]:
import os
from astropy.time import Time, TimeDelta
from IPython.display import Markdown, display, HTML

from rubin_nights import connections

## Gather connection information and authorization

In the USDF (usdf-rsp or usdf-rsp-dev) or summit RSP, the default is to connect to the services appropriate to the same RSP.

Outside of these RSPs, the default is to connect to the production USDF services at usdf-rsp.slac.stanford.edu, although the usdf-rsp-dev is available as well. Note that outside of the RSP, an RSP token will be required. See https://rsp.lsst.io/v/usdfprod/guides/auth/creating-user-tokens.html for more information about creating tokens (and note that tokens created on usdf-rsp are different than those on usdf-rsp-dev and must match the services being queried). 

In [ ]:
on_rsp = False
# Are you on an RSP?
if on_rsp:
    tokenfile = None
    site = None
# Or are you outside of an RSP? - just use USDF and your own USDF-RSP token
# See https://rsp.lsst.io/guides/auth/creating-user-tokens.html
else:
    # Substitute the location of your own tokenfile
    # If you prefer, this will also get token info from an "ACCESS_TOKEN" environment variable
    tokenfile = os.path.join(os.path.expanduser("~"), ".lsst/usdf_rsp")
    site = 'usdf'

endpoints = connections.get_clients(tokenfile = tokenfile, site=site)

The services available and their endpoints:

In [ ]:
endpoints

In [ ]:
# Example of choosing to connect to the usdf-rsp-dev services instead
endpoints_dev = connections.get_clients(tokenfile=os.path.join(os.path.expanduser("~"), ".lsst/usdf_rsp"), site='usdf-dev')
endpoints_dev

## Querying for data

Each of these services has different data. A basic query setup along the lines of "what happened on a given night" is included with most of the connection classes, and demonstrated below. 
Each connection also supports more general queries, `endpoints["xxx"].query` but the details of what to include to specify such a query depends on the service. 

More information: 
* [night report](https://usdf-rsp.slac.stanford.edu/nightreport/docs#/default/find_nightreports_reports_get)
* [narrative log](https://usdf-rsp.slac.stanford.edu/narrativelog/docs#/default/find_messages_messages_get)
* [exposure log](https://usdf-rsp.slac.stanford.edu/exposurelog/docs#/default/find_messages_messages_get)
* [Consdb Schema](https://sdm-schemas.lsst.io/) - see also [consdb technote](https://dmtn-227.lsst.io/)
    * [consdb FastAPI](https://usdf-rsp.slac.stanford.edu/consdb/docs#/default/query_consdb_query_post)
    * [consdb_TAP](https://pyvo.readthedocs.io/en/latest/dal/index.html#pyvo-tap)
* [EFD Schema](https://ts-xml.lsst.io/) - see also [lsst-efd-client documentation](https://efd-client.lsst.io/index.html)


In [ ]:
# Let's pick a day_obs to query 
# day_obs = the local calendar date of the start (sunset) of a night
day_obs = "2025-04-15"
# the day_obs is applied to the entire time from noon UTC to next-noon
t_start = Time(f"{day_obs}T12:00:00", format='isot', scale='utc')
t_end = t_start + TimeDelta(1, format='jd')
t_start.iso, t_end.iso

---
    
### Night report

The night report is a summary of the observing state. There is one for each telescope. 

In [ ]:
nightreport, html = endpoints['night_report'].query_night_report(day_obs=day_obs, telescope="Simonyi", return_html=True)
display(HTML(html))

---

### Narrative log

The narrative log is an ongoing record of comments regarding observing and the observatory state.
When paired with further information about the scriptqueue, it provides valuable insights into observatory operations.

In [ ]:
log = endpoints['narrative_log'].query_log(t_start, t_end)
# The log is indexed by time such that it can be joined with EFD data.
display(HTML(log[['component', 'message_text', 'user_id']].iloc[21:35].to_html()))

---

### Exposure log

The exposure log provides a record for annotations on particular images. It tends to be very sparsely populated.

In [ ]:
exposurelog = endpoints['exposure_log'].query_log(t_start, t_end)
# Exposure log needs to be joined with other data sources (exposure id <-> exposure time) in order to have a useful "time" value. 
exposurelog

---

### ConsDB

The consolidated database is still in development. For more information on contents, see [sdm-schemas](https://sdm-schemas.lsst.io/). 
For more information on development, see also [consdb technote](https://dmtn-227.lsst.io/). 

The TAP service is intended to be the future API for users, and is currently available on all USDF platforms (and summit?). 
The FastAPI service is available on at the USDF as well as the summit. This interface is intended to be deprecated in favor of TAP, but does perform better presently.

See also the ConsDB demo notebook in this repo. 

In [ ]:
# Let's pick a different day_obs to query - 2025-04-15 is nice but many of the visits were missing metadata
# day_obs = the local calendar date of the start (sunset) of a night
day_obs = "2025-06-20"
# the day_obs is applied to the entire time from noon UTC to next-noon
t_start = Time(f"{day_obs}T12:00:00", format='isot', scale='utc')
t_end = t_start + TimeDelta(1, format='jd')
t_start.iso, t_end.iso

In [ ]:
visits_api = endpoints['consdb'].get_visits('lsstcam', t_start, t_end, augment=False)
visits_tap = endpoints['consdb_tap'].get_visits('lsstcam', t_start, t_end, augment=False)
print(len(visits_api), len(visits_tap))

In [ ]:
short_cols = ['obs_start', 'obs_start_mjd', 's_ra', 's_dec', 'sky_rotation', 'band', 
         'airmass', 'psf_sigma_median', 'zero_point_median', 'img_type', 'target_name', 'observation_reason', 'science_program']
if len(visits_api) == 0:
    print("Did not find any visits")
else:
    display(visits_api[short_cols].iloc[200:210])

In [ ]:
from rubin_nights.augment_visits import augment_visits

In [ ]:
query = "select v.*, q.zero_point_median, q.sky_bg_median from cdb_lsstcomcam.visit1 as v, cdb_lsstcomcam.visit1_quicklook as q"
query += " where q.visit_id = v.visit_id and v.science_program = 'BLOCK-320'"
print(query)
# Either of these methods will work - query issues are more clear through TAP
#visits_lsstcomcam = endpoints['consdb_tap'].query(query)
visits_lsstcomcam = endpoints['consdb'].query(query)
visits_lsstcomcam = augment_visits(visits_lsstcomcam, "lsstcomcam")
print(f"Retrieved {len(visits_lsstcomcam)} visits") # should retrieve 1694 visits
cols = ['obs_start', 'obs_start_mjd', 's_ra', 's_dec', 'sky_rotation', 'band', 'exp_time', 'zero_point_median', 'sky_bg_median',
         'airmass', 'img_type', 'target_name', 'observation_reason', 'science_program', 'visit_gap', 'clouds']
visits_lsstcomcam[cols].tail()

---

### EFD

There is a full client for the EFD (see [efd-lsst-client](https://efd-client.lsst.io/)) but a simple, synchronous query client is provided here that replicates the basics of the API.
Finding information in the EFD can be challenging; see [the XML specifications](https://ts-xml.lsst.io/) - after identifying the CSC that is the source of the information, the commands and logentry information can be linked to the specifications for each topic.

In [ ]:
targets = endpoints['efd'].select_time_series('lsst.sal.Scheduler.logevent_target', '*', t_start, t_end, index=1)
targets.tail()

In [ ]:
prev_targets = endpoints['efd'].select_top_n('lsst.sal.Scheduler.logevent_target', '*', num=3, time_cut=t_start)
prev_targets

In [ ]:
query = "select message, traceback, salIndex from \"lsst.sal.Script.logevent_logMessage\""
query += f"where time >= \'{t_start.isot}Z\' and time <= \'{t_end.isot}Z\' and traceback != ''"
tracebacks = endpoints['efd'].query(query)
display(HTML(tracebacks[['salIndex', 'message', 'traceback']].head().to_html(escape=True)))

---

### Obs-env

This is likely uninteresting for anyone outside of summit and survey scheduling teams, but provides more details on the packages set up and in use at a given time at the summit.

In [ ]:
obsenv = endpoints['obsenv'].select_time_series('lsst.obsenv.summary', '*', t_start, t_end)
obsenv_start = endpoints['obsenv'].select_top_n('lsst.obsenv.summary', '*', num=1, time_cut=t_start)
obsenv_start